In [2]:
import numpy as np
import pandas as pd
import time
import warnings
import csv
import datetime
from datetime import datetime, date, time, timedelta

# Now let's bring in some new libraries from scipy
from scipy import stats

In [3]:
parse_dates = ['completeTime']
df_CC_CM = pd.read_csv ('CC_compliantManagement.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
df_CC_OM = pd.read_csv ('CC_orderManagement.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
onlyTS = pd.read_csv ('CC_orderManagement.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
df_FA_MI = pd.read_csv ('FA_manageInstallation.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
df_SA_PM = pd.read_csv ('SA_paymentManagement.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
df_SA_SM = pd.read_csv ('SA_serviceManagement.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
warnings.filterwarnings('ignore')

In [4]:
df_CC_CM = df_CC_CM.drop('simulated:logProbability', axis=1)
df_CC_CM = df_CC_CM.drop('concept:simulated', axis=1)
df_CC_CM = df_CC_CM.drop('startTime', axis=1)
df_CC_CM = df_CC_CM.drop('completeTime', axis=1)

df_CC_OM = df_CC_OM.drop('concept:simulated', axis=1)
df_CC_OM = df_CC_OM.drop('simulated:logProbability', axis=1)
df_CC_OM = df_CC_OM.drop('startTime', axis=1)
df_CC_OM = df_CC_OM.drop('completeTime', axis=1)

df_FA_MI = df_FA_MI.drop('simulated:logProbability', axis=1)
df_FA_MI = df_FA_MI.drop('startTime', axis=1)
df_FA_MI = df_FA_MI.drop('concept:simulated', axis=1)
df_FA_MI = df_FA_MI.drop('completeTime', axis=1)

df_SA_PM = df_SA_PM.drop('startTime', axis=1)
df_SA_PM = df_SA_PM.drop('simulated:logProbability', axis=1)
df_SA_PM = df_SA_PM.drop('concept:simulated', axis=1)
df_SA_PM = df_SA_PM.drop('completeTime', axis=1)

df_SA_SM = df_SA_SM.drop('simulated:logProbability', axis=1)
df_SA_SM = df_SA_SM.drop('concept:simulated', axis=1)
df_SA_SM = df_SA_SM.drop('startTime', axis=1)
df_SA_SM = df_SA_SM.drop('completeTime', axis=1)



In [5]:
final_df = pd.DataFrame(columns = ['case','event','time','fuelUsed','paymentTerms','status'])
a = None
logs = range(5000)
print(final_df.columns.tolist())
#counter = 50000

['case', 'event', 'time', 'fuelUsed', 'paymentTerms', 'status']


In [6]:
print(df_CC_CM.iloc[-1])

case                    4999
event    end_event_keep calm
Name: 24947, dtype: object


In [7]:
for i in logs:
    print(i)
    
    j = np.random.randint(1, 10)
    if(((df_CC_OM['event'] == 'end_event_notification unavailability') & (df_CC_OM['case'] == i)).any() == False):
        ts = onlyTS.iloc[0]['completeTime'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_CC_OM.head(7)
        #onlyTS.drop(onlyTS[:5], inplace=True)
        n = np.random.randint(1, 10)
        d1.insert(2,"time",[ts, ts + timedelta(minutes = 1), ts + timedelta(minutes = 2), ts + timedelta(minutes = 10), ts + timedelta(minutes =12), ts + timedelta(hours = n),ts + timedelta(hours = n+1)])
        #d1.insert(3,"fuelUsed",[])
        #d1.insert(4,"amount",[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN])
        #d1.insert(5,"paymentTerms",[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN])
        #d1.insert(6,"status",[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN])
        final_df = pd.concat([final_df, d1.head(7)]) #copio le righe
        df_CC_OM.drop(df_CC_OM.index[:7], inplace=True) # e le cancello dal df originale
        
        

        # ora aggiungo le righe di serviceManagement
        ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_SA_SM.head(6)
        d1.insert(2,"time",[ts, ts + timedelta(minutes = 1), ts + timedelta(minutes = 25), ts + timedelta(minutes = 26), ts + timedelta(minutes =26), ts + timedelta(minutes = 28)])
        #d1.insert(3,"fuelUsed",[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]'
        #d1.insert(4,"amount",[])
        #d1.insert(5,"paymentTerms",[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN])
        d1.insert(3,"status",[a , a,'activated', a,a,a])
        final_df = pd.concat([final_df, d1.head(6)]) #until evaluateOrderParameters
        df_SA_SM.drop(df_SA_SM.index[:6], inplace=True) 
        #df_SA_SM.reset_index(inplace = True)
        #aggiungo righe di FA_installationManagement
        ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_FA_MI.head(3)
        d1.insert(2,"time",[ts, ts + timedelta(minutes = 1), ts + timedelta(minutes = 15)])
        final_df = pd.concat([final_df, d1.head(3)])
        #final_df.reset_index(inplace = True) #metto a posto gli indici
        df_FA_MI.drop(df_FA_MI.index[:3], inplace=True)

        if(((df_SA_SM['event'] == 'solicitation from the call center') & (df_SA_SM['case'] == i)).any() == True): #se c'è un solicitation in SA, allora devo aggiungere i logs del compliant management del CC
            print('dentro')
            ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
            d1 = pd.DataFrame()
            d1 = df_CC_CM.head(4)
            n = np.random.randint(1, 24)
            start = ts
            if(((df_CC_CM['event'] == 'end_event_keep calm') & (df_CC_CM['case'] == i)).any() == False): #HO FORWARD COMPLIANT:
                start = ts + timedelta(days = 7)
                ts = start

            d1.insert(2,"time",[start, ts + timedelta(minutes = 3),ts + timedelta(minutes = 30) , ts + timedelta(hours = n)])
            #if(df_SA_PM.iloc[3].event in ['event_MESSAGE_forward compliant']):
             #   val = 'true'
            #else:
             #   val = 'false'

            #d1.insert(3,"isEligible",[a,a, a, val])
            final_df = pd.concat([final_df, d1.head(4) ])
            df_CC_CM.drop(df_CC_CM.index[:4], inplace=True)
            #final_df.reset_index(inplace = True)

            if(((final_df['event'] == 'end_event_keep calm') & (final_df['case'] == i)).any() == False): #ho forward compliant
                ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
                d1 = pd.DataFrame()
                d1 = df_SA_SM.head(2)
                n = np.random.randint(1, 24)
                d1.insert(2,"time",[ts, ts + timedelta(hours = n)])
                final_df = pd.concat([final_df, d1.head(2)])
                #final_df.reset_index(inplace = True) #metto a posto gli indici
                df_SA_SM.drop(df_SA_SM.index[:2], inplace=True)
            else:
                #final_df = final_df.append(df_SA_SM.loc[(df_SA_PM['event'] == 'evaluate order parameters') & (df_SA_PM['case'] == 0)])
                df_SA_SM.drop(df_SA_SM.index[:2], inplace=True)

        else:
            df_CC_CM.drop(df_CC_CM.loc[df_CC_CM['case']== i].index, inplace=True)

        if(((final_df['event'] == 'solicitation from the call center') & (final_df['case'] == i)).any() == True):
            if(((final_df['event'] == 'end_event_keep calm') & (final_df['case'] == i)).any() == False):
                if(((df_FA_MI['event'] == 'message_receive solicitation manage installation') & (df_FA_MI['case'] == i)).any() == True):
                    ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
                    d1 = pd.DataFrame()
                    d1 = df_FA_MI.head(2)
                    n = np.random.randint(1, 24)
                    d1.insert(2,"time",[ts, ts + timedelta(hours = n)])
                    final_df = pd.concat([final_df, d1.head(2)])
                    df_FA_MI.drop(df_FA_MI.index[:2], inplace=True)
                else:
                    ts = final_df.iloc[-1]['time'] + timedelta(minutes = 10)
                    #print(ts)
                    new1 = [i, "message_receive solicitation manage installation", ts]
                    new2 = [i, "reschedule appointments", ts + timedelta(hours = n)]
                    final_df = final_df.append(pd.Series(new1, index=final_df.columns[:len(new1)]), ignore_index=True)
                    final_df = final_df.append(pd.Series(new2, index=final_df.columns[:len(new2)]), ignore_index=True)
        
        elif((((df_FA_MI['event'] == 'message_receive solicitation manage installation') & (df_FA_MI['case'] == i)).any() == True)):
            df_FA_MI.drop(df_FA_MI.index[:2], inplace=True)
        
        
        ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_FA_MI.head(4)
        n = np.random.randint(24, 120)
        fuel =np.random.randint(1,7)
        #fuel_wrong = np.random.randit(9,15)
        #coin = np.random.randint(0,1)
        d1.insert(2,"time",[ts + timedelta(hours = n), ts + timedelta(hours = n+0.5),ts + timedelta(hours = n+0.5) , ts + timedelta(hours = n+0.6)])
        d1.insert(3,"fuelUsed",[fuel,'','',''])
        final_df = pd.concat([final_df, d1.head(4)])
        df_FA_MI.drop(df_FA_MI.index[:4], inplace=True)

        ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_SA_SM.head(3)
        d1.insert(2,"time",[ts + timedelta(minutes = 1), ts + timedelta(minutes = 2), ts + timedelta(minutes = 3)])
        d1.insert(3,"status",['in full',a,a])
        final_df = pd.concat([final_df, d1.head(3)])
        df_SA_SM.drop(df_SA_SM.index[:3], inplace=True)

        ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_CC_OM.head(2)
        d1.insert(2,"time",[ts + timedelta(minutes=1), ts + timedelta(minutes = 2)])
        final_df = pd.concat([final_df, d1.head(2)])
        df_CC_OM.drop(df_CC_OM.index[:2], inplace=True)
        if(((final_df['event'] == 'solicitation from the call center') & (final_df['case'] == i)).any() == True):
            ts = final_df.iloc[-1]['time'] + timedelta(minutes = i)
            d1 = pd.DataFrame()
            d1 = df_CC_CM.head(2)
            d1.insert(2,"time",[ts + timedelta(minutes=1), ts + timedelta(minutes = 2)])
            final_df = pd.concat([final_df, d1.head(2)])
            df_CC_CM.drop(df_CC_CM.index[:2], inplace=True)

        #d = df_SA_PM.loc[(df_SA_PM.case == i) & (df_SA_PM.event == 'event_MESSAGE_send invoice') , 'paymentTerms'].values[0]

        d1 = pd.DataFrame()
        d1 = df_SA_PM.loc[df_SA_PM['case'] == i]
        lunghezza = d1.shape[0]
        #terms = np.random.rand(1,14)
        #print(lunghezza)
        lista = []
        terms = []
        pt = np.random.randint(3,14)

        receipt = np.random.randint(1,pt-1)
        
        for t in range(lunghezza):
            if(df_SA_PM.iloc[t].event in ['event_TIMER_payment terms expire']):
                lista.append(ts+timedelta(days = pt))
                ts = ts+timedelta(days = pt)
            elif(df_SA_PM.iloc[t].event in ['event_MESSAGE_payment receipt']):
                 lista.append(ts+timedelta(days = receipt)) 
            else:
                  lista.append(ts+timedelta(minutes = receipt)) 
            #if(df_SA_PM.iloc[t].event == 'event_MESSAGE_send invoice' | df_SA_PM.iloc[t].event == "event_MESSAGE_send solicitation with new amount"):
            if(df_SA_PM.iloc[t].event in ['event_MESSAGE_send invoice', 'event_MESSAGE_send solicitation with new amount']):
                terms.append(pt)
            else:
                terms.append('')
        #for b in terms:
         #   print(b)        
        d1.insert(2,"time", lista)
        d1.insert(3,"paymentTerms", terms)
        #print(d1.shape[0])
        final_df = pd.concat([final_df, d1.head(lunghezza)])
        df_SA_PM.drop(df_SA_PM.loc[df_SA_PM['case']== i].index, inplace=True)
        #print(final_df.shape[0])
        #final_df.loc[(final_df.case == i) & (final_df.event == 'event_TIMER_payment terms expire') , 'Time'] = final_df.loc[(final_df.case == i) & (final_df.event == 'event_TIMER_payment terms expire') , 'Time'] + timedelta(days = d)
        #final_df.loc[(final_df.case == i) & (final_df.event == 'recalculate amount') , 'Time'] = df1.loc[(df1.case == i) & (final_df.event == 'recalculate amount') , 'Time'] + timedelta(days = d)
        #final_df.loc[(final_df.case == i) & (final_df.event == 'event_MESSAGE_send solicitation with new amount') , 'Time'] = final_df.loc[(df1.case == i) & (df1.event == 'event_MESSAGE_send solicitation with new amount') , 'Time'] + timedelta(days = d)
        #final_df.loc[(final_df.case == i) & (final_df.event == 'event_MESSAGE_payment receipt') , 'Time'] = final_df.loc[(final_df.case == i) & (df1.event == 'event_MESSAGE_payment receipt') , 'Time'] + timedelta(days = d)
        #final_df.loc[(final_df.case == i) & (final_df.event == 'end_event_') , 'Time'] = final_df.loc[(df1.case == i) & (final_df.event == 'end_event_') , 'Time'] + timedelta(days = d)

    else:
        ts = onlyTS.iloc[0]['completeTime'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_CC_OM.head(4)
        d1.insert(2,"time", [ts, ts+ timedelta(minutes = 1), ts +timedelta(minutes = 10), ts +timedelta(minutes = 11)])
        final_df = pd.concat([final_df, d1.head(4)])
        df_CC_OM.drop(df_CC_OM.index[:4], inplace=True)
        df_CC_CM.drop(df_CC_CM.loc[df_CC_CM['case']== i].index, inplace=True)
        df_FA_MI.drop(df_FA_MI.loc[df_FA_MI['case']== i].index, inplace=True)
        df_SA_PM.drop(df_SA_PM.loc[df_SA_PM['case']== i].index, inplace=True)
        df_SA_SM.drop(df_SA_SM.loc[df_SA_SM['case']== i].index, inplace=True)
    
    
       #if(i == counter):
        #tosave = pd.DataFrame()
        #tosave = final_df
        #counter = counter + 10000
        #final_df.set_index('case', inplace = True)
        #final_df.to_csv('finalDF' + str(i) + '.csv')
        #final_df = pd.DataFrame()
    
    




0
dentro
1
2
3
4
dentro
5
6
7
8
dentro
9
10
11
12
13
dentro
14
15
16
17
18
19
20
21
22
dentro
23
dentro
24
25
26
27
28
dentro
29
30
31
32
dentro
33
34
35
dentro
36
dentro
37
38
39
40
41
dentro
42
43
44
dentro
45
46
dentro
47
dentro
48
49
50
dentro
51
dentro
52
53
54
55
dentro
56
57
58
59
60
61
62
63
64
65
dentro
66
67
68
69
70
71
dentro
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
dentro
92
93
94
dentro
95
96
dentro
97
dentro
98
99
100
101
dentro
102
103
104
dentro
105
106
107
108
dentro
109
110
111
112
113
114
dentro
115
dentro
116
117
dentro
118
119
120
121
dentro
122
123
124
125
126
127
128
129
130
dentro
131
132
133
134
135
136
137
dentro
138
139
dentro
140
dentro
141
dentro
142
143
144
145
146
147
dentro
148
149
dentro
150
151
152
153
154
155
156
157
dentro
158
159
160
161
dentro
162
dentro
163
164
165
dentro
166
167
168
169
170
171
172
173
174
dentro
175
176
177
178
179
180
181
182
183
dentro
184
dentro
185
dentro
186
187
188
189
190
dentro
191
dentro
192
193
dentr

In [8]:
#final_df = final_df.drop('simulated:logProbability', axis=1)
#final_df = final_df.drop('concept:simulated', axis=1)
final_df.set_index('case', inplace = True)
final_df.to_csv('finalDFPULITI.csv')
print('a')

a


In [9]:
print(final_df.iloc[-1])
print(final_df.shape[0])

event                     SA_PM_end_event_
time            2022-11-03 03:39:01.177000
fuelUsed                               NaN
paymentTerms                              
status                                 NaN
Name: 4999, dtype: object
100338
